In [ ]:
import io
from PIL import Image
from ultralytics import YOLO
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials

# --- 1. 설정 (사용자 정보 입력) ---
# Azure Custom Vision 정보
PREDICTION_KEY = 
ENDPOINT = "https://8aiteam5pilldetect-prediction.cognitiveservices.azure.com/"
PROJECT_ID = "9f1ab1d6-b2c3-4d0a-8125-66457be87432"
MODEL_NAME = "Iteration6" # 예: "Iteration"

# YOLO 모델 경로
YOLO_MODEL_PATH = "best_detector_v1.pt"

# 입력 이미지
INPUT_IMAGE_PATH = "D:/AiSchool/1stProject/New Data/KakaoTalk_20251116_155740678.jpg"

# --- 2. 모델 및 클라이언트 초기화 ---
# YOLO 모델 로드
yolo_model = YOLO(YOLO_MODEL_PATH)

# Azure Custom Vision 예측 클라이언트 인증
credentials = ApiKeyCredentials(in_headers={"Prediction-Key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(ENDPOINT, credentials)

# --- 3. 메인 파이프라인 실행 ---
print(f"이미지 처리 시작: {INPUT_IMAGE_PATH}")

# 원본 이미지 로드 (PIL 사용)
original_image = Image.open(INPUT_IMAGE_PATH)

# 1단계: YOLO로 알약 탐지
results = yolo_model(original_image,max_det=1)

pill_results = [] # 최종 결과를 저장할 리스트

# 2단계: 탐지된 알약 영역 자르기
# results[0].boxes.xyxy 는 [x1, y1, x2, y2] 좌표 리스트를 포함합니다.
for box in results[0].boxes:
    # 좌표 추출
    coords = box.xyxy[0].tolist() # [x1, y1, x2, y2]
    
    # PIL 이미지에서 해당 영역 자르기
    cropped_pill = original_image.crop(coords)
    print(f"알약 탐지됨. 좌표: {coords}")

    # 3단계: Custom Vision으로 분류
    # 이미지를 바이트로 변환
    with io.BytesIO() as output:
        cropped_pill.save(output, format="JPEG")
        image_bytes = output.getvalue()

    # Azure API 호출
    cv_results = predictor.classify_image(
        PROJECT_ID,
        MODEL_NAME,
        image_bytes
    )

    # 4단계: 결과 저장
    # 가장 확률이 높은 예측 결과
    top_prediction = cv_results.predictions[0]
    result_info = {
        "box_coords": coords,
        "pill_type": top_prediction.tag_name,
        "confidence": f"{top_prediction.probability * 100:.2f}%"
    }
    pill_results.append(result_info)

# --- 4. 최종 결과 출력 ---
print("\n--- 최종 알약 식별 결과 ---")
if not pill_results:
    print("이미지에서 알약을 탐지하지 못했습니다.")
else:
    for i, res in enumerate(pill_results):
        print(f"  [알약 {i+1}]")
        print(f"    종류: {res['pill_type']} (정확도: {res['confidence']})")
        print(f"    위치: {res['box_coords']}")

이미지 처리 시작: D:/AiSchool/1stProject/New Data/KakaoTalk_20251116_155740678.jpg

0: 480x640 1 Pill, 82.7ms
Speed: 135.2ms preprocess, 82.7ms inference, 10.5ms postprocess per image at shape (1, 3, 480, 640)
알약 탐지됨. 좌표: [2122.687255859375, 1337.12890625, 2499.92578125, 1936.9793701171875]

--- 최종 알약 식별 결과 ---
  [알약 1]
    종류: 까스앤프리츄정80밀리그램 (정확도: 90.29%)
    위치: [2122.687255859375, 1337.12890625, 2499.92578125, 1936.9793701171875]
